In [2]:
import os
import shutil

def copy_gh_files(source_folder, destination_folder):
    # Walk through all subfolders in the source folder
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            # Check if the file has a .gh extension
            if file.endswith(".gh"):
                # Construct the full file paths
                source_file = os.path.join(root, file)
                destination_file = os.path.join(destination_folder, file)
                
                try:
                    # Copy the file from the source to the destination
                    shutil.copy2(source_file, destination_file)
                    print(f"Copied: {source_file} -> {destination_file}")
                except IOError as e:
                    print(f"Error copying file: {source_file}. {str(e)}")
                except Exception as e:
                    print(f"Unexpected error occurred while copying file: {source_file}. {str(e)}")
src = r"C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\Junichira GrasshopperHowtos-master"
dst = r"C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\ExtractionEnvironments\240318-initial parsing\03-GH_Files\03a-Raw"


In [1]:
import numpy as np
import networkx as nx
def one_hot_on_attribute(G, attribute:str):
    """
    Create a one hot encoding where the mapping dictionary determines the column index of the one hot encoding. 
    The rows represent each node in the graph. The attribute is the node information that we want to categorise.
    
    Parameters:
    G (nx.Graph or nx.DiGraph): The graph to be processed.
    attribute (str): The attribute of the node to be categorised.
    
    Returns:
    np.array: A one hot encoding of the node attributes.
    
    Example:
     G = nx.Graph()
     blue, orange, green = "#1f77b4", "#ff7f0e","#2ca02c"
     G.add_nodes_from([(1, {"color": blue}),
                       (2, {"color": orange}),
                       (3, {"color": blue}),
                       (4, {"color": green})])
     G.add_edges_from([(1, 2),(2, 3),(1, 3),(3, 4)])
     one_hot_encoding = one_hot_on_attribute(G, 'color')
     print(one_hot_encoding)
    [[1. 0. 0.]
     [0. 1. 0.]
     [1. 0. 0.]
     [0. 0. 1.]]
    """
    set_attributes = {v:i for i, v in enumerate(set(nx.get_node_attributes(G, 'color').values()))}
    one_hot_idx = np.array([set_attributes[v] for v in nx.get_node_attributes(G, attribute).values()])
    one_hot_encoding = np.zeros((one_hot_idx.size, len(set_attributes)))
    one_hot_encoding[np.arange(one_hot_idx.size), one_hot_idx] = 1
    return one_hot_encoding